In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import numpy as np
import math
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import euclidean_distances
from scipy import stats
import random
import warnings
warnings.filterwarnings("ignore")
get_ipython().magic('matplotlib inline')

C:\Users\Maher\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# Load in the data and drop extraneous columns
df = pd.read_csv('NeuronDataMaster2.csv')
df=df.dropna(axis=1, how='all')


FileNotFoundError: File b'NeuronDataMaster2.csv' does not exist

In [ ]:
#Defining a sort function to be used later
def Sort(DataType, df):
    df=df.sort_values(DataType)
    return df

In [ ]:
#Making dataframes to hold the principal cells and the interneurons 
Principal_Cells = df.where(df.Primary_Cell_Class == 'principal cell').dropna()
Interneuron_Cells = df.where(df.Primary_Cell_Class == 'interneuron').dropna()

In [ ]:
print('Principal Cells average Number of Branches: ', Principal_Cells.Number_Branches.mean())
print('Interneuron Cells average Number of Branches: ', Interneuron_Cells.Number_Branches.mean())
print('Principal Cells Number of Branches Standard Deviation: ', Principal_Cells.Number_Branches.std())
print('Interneuron Cells Number of Branches Standard Deviation: ', Interneuron_Cells.Number_Branches.std())
print(' ')
print('Principal Cells average Volume: ', Principal_Cells.Total_Volume.mean())
print('Interneuron Cells average Volume: ', Interneuron_Cells.Total_Volume.mean())
print('Principal Cells Volume Standard Deviation: ', Principal_Cells.Total_Volume.std())
print('Interneuron Cells Volume Standard Deviation: ', Interneuron_Cells.Total_Volume.std())
print(' ')
print('Principal Cells average Surface: ', Principal_Cells.Total_Surface.mean())
print('Interneuron Cells average Surface: ', Interneuron_Cells.Total_Surface.mean())
print('Principal Cells Surface Standard Deviation: ', Principal_Cells.Total_Surface.std())
print('Interneuron Cells Surface Standard Deviation: ', Interneuron_Cells.Total_Surface.std())

In [ ]:
#In order to compare the mean value of two populations, we must use a hypothesis test. In this test, we use the difference of the 
#two calculated means and an adjusted standard deviation to see if 0 is within the interval of 90% confidence. If 0 is contained
#in the interval, then the test will return false, indicating that no conclusion can be reached (i.e. we cannot reject our null
#hypothesis that the are the same).Otherwise, the test indicates there is a significant difference between the two populations
def Hypothesis_Test(data1, data2 ):
    #All of our Hypothesis tests will use a signifigance level of 0.05 or 90% confidence. 
    size1 = len(data1.index)
    size2 = len(data2.index)
    Delta = data1.mean()-data2.mean()
    Std = math.pow(math.pow(data1.std(),2)/size1+math.pow(data2.std(),2)/size2,.5)
    if (((Delta - 2.132*Std>0)and(Delta +2.132*Std>0))or((Delta - 2.132*Std<0)and(Delta +2.132*Std<0))):
        return True
    else:
        return False

In [ ]:
print('Significant Difference in Number of Branches: ', Hypothesis_Test(Principal_Cells.Number_Branches, Interneuron_Cells.Number_Branches))
print('Significant Difference in Volume: ',Hypothesis_Test(Principal_Cells.Total_Volume, Interneuron_Cells.Total_Volume) )
print('Significant Difference in Surface: ', Hypothesis_Test(Principal_Cells.Total_Surface, Interneuron_Cells.Total_Surface))

In [ ]:
#For the next 3 cells, we compare all the types of secondary class cells and see if there is a signifigant difference between the
#two being compared. Note: the order of the comparison does not affect the result, i.e. A compared to B is the same as B compared
# to A. As a result, our loops ony do one comparison be combination
Types = df.Secondary_Cell_Class.unique()
for n in range(0, len(Types)):
    data1 = df.where(df.Secondary_Cell_Class == Types[n]).dropna()
    for m in range(n+1, len(Types)):
        data2 = df.where(df.Secondary_Cell_Class == Types[m]).dropna()
        #print('Significant differences in Number of Branches', Types[n],' ', Types[m],' : ',Hypothesis_Test(data1.Number_Branches,data2.Number_Branches))
        if((Hypothesis_Test(data1.Number_Branches,data2.Number_Branches))):
            print('Significant differences in Number of Branches', Types[n],' ', Types[m])
        else: 
            print('No Significant differences in Number of Branches', Types[n],' ', Types[m])

In [ ]:
Types = df.Secondary_Cell_Class.unique()
for n in range(0, len(Types)):
    data1 = df.where(df.Secondary_Cell_Class == Types[n]).dropna()
    for m in range(n+1, len(Types)):
        data2 = df.where(df.Secondary_Cell_Class == Types[m]).dropna()
        #print('Significant differences in Volume', Types[n],' ', Types[m],' : ',Hypothesis_Test(data1.Total_Volume,data2.Total_Volume))
        if(Hypothesis_Test(data1.Total_Volume,data2.Total_Volume)):
            print('Significant differences in Volume', Types[n],' ', Types[m])
        else:
            print('No Significant differences in Volume', Types[n],' ', Types[m])

In [ ]:
Types = df.Secondary_Cell_Class.unique()
for n in range(0, len(Types)):
    data1 = df.where(df.Secondary_Cell_Class == Types[n]).dropna()
    for m in range(n+1, len(Types)):
        data2 = df.where(df.Secondary_Cell_Class == Types[m]).dropna()
        #print('Significant differences in Surface', Types[n],' ', Types[m],' : ',Hypothesis_Test(data1.Total_Surface,data2.Total_Surface))
        if(Hypothesis_Test(data1.Total_Surface,data2.Total_Surface)):
            print('Significant differences in Surface', Types[n],' ', Types[m])
        else:
            print('No Significant differences in Surface', Types[n],' ', Types[m])

In [ ]:
#Since running through ~6000 results for one comparison, we defined a compare function for all three qualities we were hunting
def compare_secondary_branches(type1,type2,df):
    data1 = df.where(df.Secondary_Cell_Class == type1).dropna()
    data2 = df.where(df.Secondary_Cell_Class == type2).dropna()
    if(Hypothesis_Test(data1.Number_Branches,data2.Number_Branches)):
            print('Significant differences in Branches' , type1,' ', type2)
    else:
            print('No Significant differences in Branches', type1,' ', type2)
def compare_secondary_volume(type1,type2,df):
    data1 = df.where(df.Secondary_Cell_Class == type1).dropna()
    data2 = df.where(df.Secondary_Cell_Class == type2).dropna()
    if(Hypothesis_Test(data1.Total_Volume,data2.Total_Volume)):
            print('Significant differences in Volume' , type1,' ', type2)
    else:
            print('No Significant differences in Volume', type1,' ', type2) 
def compare_secondary_surface(type1,type2,df):
    data1 = df.where(df.Secondary_Cell_Class == type1).dropna()
    data2 = df.where(df.Secondary_Cell_Class == type2).dropna()
    if(Hypothesis_Test(data1.Total_Surface,data2.Total_Surface)):
            print('Significant differences in Surface' , type1,' ', type2)
    else:
            print('No Significant differences in Surface', type1,' ', type2) 

In [ ]:
#Now we set up to determine if any staining, slicing direction, or thickness provide less data than the others. First we find the
#amount of fields nor reported in the data and make a new dataframe that contains the amount of not reports and the stain, slicing 
#direction, and sliing thickness
not_reported_np = np.zeros(shape=(len(df),4))
not_reported = pd.DataFrame(not_reported_np)
for n in range(0, len(df)):
    number_not_reported = 0
    for m in range(0,53):
        if(df.iat[n,m]=='Not reported'):
            number_not_reported += 1
    not_reported.set_value(n,0,number_not_reported)
not_reported[1]=df.Staining_Method
not_reported[2]=df.Slicing_Direction
not_reported[3]=df.Slice_Thickness
not_reported.columns = ['Number_not_reported', 'Stain','Slice_Direction', 'Slice_Thickness']
not_reported = Sort('Stain', not_reported)

In [ ]:
Types = df.Staining_Method.unique()
#We now make a dataframe witht the stain, average amount not reported, and the standard deviation of the not reported. We then take
#the minimum in order to find the stain with the least non-reports
Averages_np = np.zeros(shape=(len(Types),3))
Averages_stain = pd.DataFrame(Averages_np)
for n in range(0,len(Types)):
    data = not_reported.where(not_reported.Stain == Types[n])
    Averages_stain.set_value(n,0,data.mean())
    Averages_stain.set_value(n,1,data.std())
Averages_stain[2]=Types
Averages_stain.columns = ['Average', 'Standard_Deviation', 'Stain']


In [ ]:
Minimum_stain = Averages.where(Averages.Average ==Averages.Average.min()).dropna()
Minimum_stain
#So biotinylated dextran amine gives the least amount of non-reports

In [ ]:
for n in range(0, len(Types)):
    data1 = not_reported.where(not_reported.Stain == Types[n]).dropna()
    for m in range(n+1, len(Types)):
        data2 = not_reported.where(not_reported.Stain == Types[m]).dropna()
        if(Hypothesis_Test(data1.Number_not_reported,data2.Number_not_reported)):
            print('Significant differences amount reported', Types[n],' ', Types[m])
        else:
            print('No Significant differences in amounr reported', Types[n],' ', Types[m])

In [ ]:
#Again, since there are many results, we defince search functions in order to quickly find the comparison wanted
def compare_stain(type1,type2,df):
    data1 = df.where(df.Stain == type1).dropna()
    data2 = df.where(df.Stain == type2).dropna()
    if(Hypothesis_Test(data1.Number_not_reported,data2.Number_not_reported)):
            print('Significant differences in number reported' , type1,' ', type2)
    else:
            print('No Significant differences in number reported', type1,' ', type2)

In [ ]:
#We then apply this to the slicing direction and slicing thickness
Types = df.Slicing_Direction.unique()
Averages_np = np.zeros(shape=(len(Types),3))
Averages_slice_direction = pd.DataFrame(Averages_np)
for n in range(0,len(Types)):
    data = not_reported.where(not_reported.Slice_Direction == Types[n])
    Averages_slice_direction.set_value(n,0,data.mean())
    Averages_slice_direction.set_value(n,1,data.std())
Averages_slice_direction[2]=Types
Averages_slice_direction.columns = ['Average', 'Standard_Deviation', 'Slice_Direction']

In [ ]:
Minimum_slicing_direction = Averages_slice_direction.where(Averages_slice_direction.Average ==Averages_slice_direction.Average.min()).dropna()
Minimum_slicing_direction
#So oblique horizontal cuts gives the least amount of non-reports

In [ ]:
for n in range(0, len(Types)):
    data1 = not_reported.where(not_reported.Slice_Direction == Types[n]).dropna()
    for m in range(n+1, len(Types)):
        data2 = not_reported.where(not_reported.Slice_Direction == Types[m]).dropna()
        if(Hypothesis_Test(data1.Number_not_reported,data2.Number_not_reported)):
            print('Significant differences amount reported', Types[n],' ', Types[m])
        else:
            print('No Significant differences in amount reported', Types[n],' ', Types[m])

In [ ]:
def compare_slicing_direction(type1,type2,df):
    data1 = df.where(df.Slice_Direction == type1).dropna()
    data2 = df.where(df.Slice_Direction == type2).dropna()
    if(Hypothesis_Test(data1.Number_not_reported,data2.Number_not_reported)):
            print('Significant differences in number reported' , type1,' ', type2)
    else:
            print('No Significant differences in number reported', type1,' ', type2)

In [ ]:
#We treat the thickness of the cuts as a categorical dataset becuse the values for the thickness is grouped into bins (caused by rounding)
#, and as a result, the accuracy of a linear regression is questionable
Types = df.Slice_Thickness.unique()
Averages_np = np.zeros(shape=(len(Types),3))
Averages_slice_thickness = pd.DataFrame(Averages_np)
for n in range(0,len(Types)):
    data = not_reported.where(not_reported.Slice_Thickness == Types[n])
    Averages_slice_thickness.set_value(n,0,data.mean())
    Averages_slice_thickness.set_value(n,1,data.std())
Averages_slice_thickness[2]=Types
Averages_slice_thickness.columns = ['Average', 'Standard_Deviation', 'Slice_Direction']

In [ ]:
Minimum_slicing_thickness = Averages_slice_thickness.where(Averages_slice_thickness.Average ==Averages_slice_thickness.Average.min()).dropna()
Minimum_slicing_thickness
#So 65 thick cuts gives the least amount of non-reports

In [ ]:
for n in range(0, len(Types)):
    data1 = not_reported.where(not_reported.Slice_Thickness == Types[n]).dropna()
    for m in range(n+1, len(Types)):
        data2 = not_reported.where(not_reported.Slice_Thickness == Types[m]).dropna()
        if(Hypothesis_Test(data1.Number_not_reported,data2.Number_not_reported)):
            print('Significant differences amount reported', Types[n],' ', Types[m])
        else:
            print('No Significant differences in amount reported', Types[n],' ', Types[m])

In [ ]:
def compare_slicing_thickness(type1,type2,df):
    data1 = df.where(df.Slice_Thickness == type1).dropna()
    data2 = df.where(df.Slice_Thickness == type2).dropna()
    if(Hypothesis_Test(data1.Number_not_reported,data2.Number_not_reported)):
            print('Significant differences in number reported' , type1,' ', type2)
    else:
            print('No Significant differences in number reported', type1,' ', type2)

In [ ]:
Types = df.Species_Name.unique()
for n in range(0,len(Types)):
    data1 = df.where(df.Species_Name==Types[n])
    data1 = data1.where(data1.Primary_Brain_Region == 'hippocampus')
    for m in range(n+1,len(Types)):
        data2 = df.where(df.Species_Name==Types[m])
        data2 = data2.where(data2.Primary_Brain_Region == 'hippocampus')
        if(Hypothesis_Test(data1.Total_Volume,data2.Total_Volume)):
            print('Significant differences in Volume', Types[n],' ', Types[m])
        else:
            print('No Significant differences in Volume', Types[n],' ', Types[m])

In [ ]:
def compare_species(type1,type2,df):
    data1 = df.where(df.Species_Name == type1).dropna()
    data1 = data1.where(data1.Primary_Brain_Region == 'hippocampus')
    data2 = df.where(df.Species_Name == type2).dropna()
    data2 = data2.where(data2.Primary_Brain_Region == 'hippocampus')
    if(Hypothesis_Test(data1.Total_Volume,data2.Total_Volume)):
            print('Significant differences in Volume' , type1,' ', type2)
    else:
            print('No Significant differences in Volume', type1,' ', type2)

In [ ]:
# Classification function to determine cell class, brain region, and species name by using PCA on morphology data
def neuronClassifier(predictors, outputLabel):
    predictDf = df[predictors]
    
    # Standardization to account for different scales of the variables
    predictDf_standardized = StandardScaler().fit_transform(predictDf)

    outputVector = df[outputLabel]

    # PCA to determine the 3 principal components and reduce dimensionality
    sklearn_pca = PCA(n_components=3)
    Y_sklearn = sklearn_pca.fit_transform(predictDf_standardized)
    reducedDf = pd.DataFrame(Y_sklearn)
    
    # Split the data into testing and training data
    testingIndices = []
    for x in range(100):
        randonum = random.randint(0,len(df.index))
        while randonum in testingIndices:
            randonum = random.randint(0,len(df.index))
        testingIndices.append(randonum)

    trainingIndices = [x for x in range(0,len(df.index)) if x not in testingIndices]
    testingSet = reducedDf.iloc[testingIndices,:]
    trainingSet = reducedDf.iloc[trainingIndices,:]

    testLabels = list(outputVector[testingIndices])
    trainLabels = list(outputVector[trainingIndices])
    
    # Generate predictions via majority vote of the 15 nearest neighbors in the training set 
    predictionVector = []
    for i, testInd in enumerate(range(len(testingSet.index))):
        print(i)
        testRow = testingSet.iloc[testInd,:]
        distanceVector = []
        for trainInd in range(len(trainingSet.index)):
            trainRow = trainingSet.iloc[trainInd,:]
            distance = float(euclidean_distances(testRow, trainRow)[0][0])
            distanceVector.append((trainInd, distance))
        distanceVector = sorted(distanceVector, key = lambda x: x[1])
        nearestNeighbors = [x[0] for x in distanceVector[:15]]
        nearestNeighborLabels = []
        for num in nearestNeighbors:
            label = trainLabels[num]
            nearestNeighborLabels.append(label)
        prediction = stats.mode(nearestNeighborLabels)[0][0]
        predictionVector.append(prediction)
    
    # Compile the results
    resultDf = pd.DataFrame(index = testingSet.index, columns = [outputLabel, 'Prediction'])
    originalTestingDf = df.iloc[testingSet.index,:]
    resultDf['Primary Cell Class'] = originalTestingDf['Primary Cell Class']
    resultDf['Prediction'] = predictionVector
    
    count = 0
    for r in list(range(len(resultDf.index))):
        if resultDf.iloc[r,0] == resultDf.iloc[r,1]:
            count +=1
    
    # Calculate accuracy
    accuracy = count / len(resultDf.index)
    return accuracy

In [ ]:
# Test classification algorithm for species name, primary cell class, and primary brain region
predictors = ['Soma Surface','Number of Stems', 'Number of Bifurcations', 'Number of Branches', 'Overall Width', 'Overall Height', 'Overall Depth', 'Average Diameter', 'Total Length', 'Total Surface', 'Total Volume', 'Max Euclidean Distance', 'Max Path Distance', 'Max Branch Order', 'Average Contraction', 'Total Fragmentation', 'Partition Asymmetry', 'Average Rall\'s Ratio', 'Average Bifurcation Angle Local', 'Average Bifurcation Angle Remote', 'Fractal Dimension']

speciesAccuracy = neuronClassifier(predictors, 'Species Name')
cellClassAccuracy = neuronClassifier(predictors, 'Primary Cell Class')
brainRegionAccuracy = neuronClassifier(predictors, 'Primary Brain Region')


In [ ]:
# Classification Results
print('Species were classified with an accuracy of ' + str(speciesAccuracy))
print('Primary Cell Classes were classified with an accuracy of ' + str(cellClassAccuracy))
print('Primary Brain Regions were classified with an accuracy of ' + str(brainRegionAccuracy))